In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [2]:
datos = pd.read_csv("C:\\Users\\crist\\Documents\\UNIVERSIDAD\\Semestre IX\\MachineLearning\\consignas\\RepoEntregaTres\\dataset_salariosLimpio.csv",
                              delimiter= ",", decimal= ".")

In [3]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30042 entries, 0 to 30041
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30042 non-null  int64 
 1   workclass       30042 non-null  object
 2   education       30042 non-null  object
 3   marital_status  30042 non-null  object
 4   occupation      30042 non-null  object
 5   relationship    30042 non-null  object
 6   race            30042 non-null  object
 7   sex             30042 non-null  object
 8   hours_per_week  30042 non-null  int64 
 9   native_country  30042 non-null  object
 10  income          30042 non-null  object
dtypes: int64(2), object(9)
memory usage: 2.5+ MB


# 3 Data recode

Aqui va lo de balancear el dataset, tambian lo de cambio de nombres de las variables, o en clean, (me cuesta mirar donde ubicar el rename de las variables) tabien se vuelan aqui las variables que presenten 0 en la tabla de relación

# Es neceasario balancear?

Se comprueba el conteo de los valores de salida del dataset

In [10]:
datos["income"].value_counts()

<=50K    22558
>50K      7484
Name: income, dtype: int64

Se nota el desbalance y se procede con la tecnica de la calculadora revisar que tendremos que restar un error de 1.27% al final

In [4]:
indices_menores = datos[datos.income=="<=50K"].sample(7484).index
datos_menores   = datos.iloc[indices_menores,]
indices_mayores = datos[datos.income==">50K"].index
datos_mayores   = datos.iloc[indices_mayores,]

datos_analisis = pd.concat([datos_menores, datos_mayores])
datos_analisis = datos_analisis.sample(len(datos_analisis))
datos_analisis.reset_index(inplace=True)
datos_analisis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14968 entries, 0 to 14967
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           14968 non-null  int64 
 1   age             14968 non-null  int64 
 2   workclass       14968 non-null  object
 3   education       14968 non-null  object
 4   marital_status  14968 non-null  object
 5   occupation      14968 non-null  object
 6   relationship    14968 non-null  object
 7   race            14968 non-null  object
 8   sex             14968 non-null  object
 9   hours_per_week  14968 non-null  int64 
 10  native_country  14968 non-null  object
 11  income          14968 non-null  object
dtypes: int64(3), object(9)
memory usage: 1.4+ MB


In [11]:
datos_analisis["income"].value_counts()

>50K     7484
<=50K    7484
Name: income, dtype: int64

# Se exporta el dataset balanceado para montar en el modelo

In [ ]:
datos_analisis.to_csv('data_analisis.csv', index=False)

# Analísis de las variables de entrada para determinar relación con la variable de salida

---
***Variables categóricas***

---

---
***Variable workclass frente a income***

---

In [6]:
pd.crosstab(datos_analisis.income, datos_analisis.workclass)

workclass,Federal-gov,Local-gov,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay
income,,,,,,,
<=50K,201,489,5766,137,573,314,4
>50K,364,608,4866,591,711,344,0


---
***Variable education frente a income***

---

In [7]:
pd.crosstab(datos_analisis.income, datos_analisis.education)

education,10th,11th,12th,1st-4th,5th-6th,7th-8th,9th,Assoc-acdm,Assoc-voc,Bachelors,Doctorate,HS-grad,Masters,Preschool,Prof-school,Some-college
income,,,,,,,,,,,,,,,,
<=50K,249,314,136,50,69,158,149,265,326,964,42,2664,212,16,44,1826
>50K,58,59,28,6,12,35,25,256,343,2118,278,1613,915,0,404,1334


---
***Variable marital_status frente a income***

---

In [15]:
pd.crosstab(datos_analisis.income, datos_analisis.marital_status)

marital_status,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed
income,,,,,,,
<=50K,1212,7,2527,111,3047,296,227
>50K,452,10,6375,31,468,66,76


---
***Variable occupation frente a income***

---

In [8]:
pd.crosstab(datos_analisis.income, datos_analisis.occupation)

occupation,Adm-clerical,Armed-Forces,Craft-repair,Exec-managerial,Farming-fishing,Handlers-cleaners,Machine-op-inspct,Other-service,Priv-house-serv,Prof-specialty,Protective-serv,Sales,Tech-support,Transport-moving
income,,,,,,,,,,,,,,
<=50K,1047,2,972,683,288,455,559,1010,42,787,152,881,220,386
>50K,498,1,906,1927,114,83,245,132,1,1807,210,963,278,319


---
***Variable relationship frente a income***

---

In [9]:
pd.crosstab(datos_analisis.income, datos_analisis.relationship)

relationship,Husband,Not-in-family,Other-relative,Own-child,Unmarried,Wife
income,,,,,,
<=50K,2219,2268,307,1470,965,255
>50K,5663,817,35,64,213,692
